## cut images

In [2]:
from gliopath.utils.slicer import tile_slides
import os
os.chdir('F:/workspace/pathology/gigapath')

FileNotFoundError: [Errno 2] No such file or directory: 'D:/workspace/pathology/gigapath'

In [ ]:
slide_paths = ['model/sample_data/PROV-000-000001.ndpi',
               'model/sample_data/PROV-000-000002.ndpi',
               'model/sample_data/PROV-000-000003.ndpi']
output_dir = 'output'

In [ ]:
tile_slides(slide_paths, save_dir=output_dir, level=1)

## run tile inference model

In [ ]:
from gliopath.models.load import giga_tile_enc
from gliopath.train.gadget import TileDataset

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from glob import glob
import os

os.chdir('F:/workspace/pathology/gigapath')
tile_encoder = giga_tile_enc(path='model/pub/')

In [ ]:
tile_encoder = tile_encoder.cuda()
tile_encoder.eval()
batch_size = 128
transform = transforms.Compose(
    [
        transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])

In [20]:
# get the folder names of folder with enough png files and csv files 
chemises_image = []

for chemise_image in os.listdir('output/tiles/'):
    n_csv = len([f for f in os.listdir('output/tiles/' + chemise_image) if f.lower().endswith('.csv')])
    n_png = len([f for f in os.listdir('output/tiles/' + chemise_image) if f.lower().endswith('.png')])
    if n_csv > (-1) and n_png > 0:
        chemises_image.append('output/tiles/' + chemise_image + '/')

In [ ]:
# 使用 with torch.no_grad() 替代装饰器
with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=torch.float16):
        
        for chemise_image in tqdm(chemises_image, desc='Slide Progress', leave=True):
            tile_dl = DataLoader(TileDataset(glob(chemise_image + '*.png', recursive=False), transform=transform), batch_size=batch_size, shuffle=False)
            tile_embeds = {'tile_embeds': [], 'coords': []}
            
            for batch in tqdm(tile_dl, desc=chemise_image, leave=False):
                tile_embeds['tile_embeds'].append(tile_encoder(batch['img'].cuda()).detach().cpu())
                tile_embeds['coords'].append(batch['coords'])

            slide_embeds = {k: torch.cat(v) for k, v in tile_embeds.items()}
            torch.save(slide_embeds, 'output/tiles/' + chemise_image + '.pt')

## [note] this script file does not contain tile encoder fine-tuning